In [ ]:
import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


ModuleNotFoundError: No module named 'numpy'

In [ ]:
metadata_path = "HAM10000_metadata.csv"
image_dir = "HAM10000_images" 

In [9]:
df = pd.read_csv(metadata_path)

NameError: name 'pd' is not defined

In [13]:
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_dir, x + ".jpg"))

In [11]:
label_map = {
    'akiec': 'Actinic keratoses',
    'bcc': 'Basal cell carcinoma',
    'bkl': 'Benign keratosis-like lesions',
    'df': 'Dermatofibroma',
    'mel': 'Melanoma',
    'nv': 'Melanocytic nevi',
    'vasc': 'Vascular lesions'
}

In [12]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)

NameError: name 'train_test_split' is not defined

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

NameError: name 'ImageDataGenerator' is not defined

In [17]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='dx',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size=32
)

Found 4772 validated image filenames belonging to 7 classes.


/Users/atulpal/Downloads/Skin-Disease-Prediction-main 2/env/lib/python3.9/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 3240 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [19]:

import json

In [20]:

with open('models/class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)

In [21]:
val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='dx',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size=32
)

Found 1190 validated image filenames belonging to 7 classes.


/Users/atulpal/Downloads/Skin-Disease-Prediction-main 2/env/lib/python3.9/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 813 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

2025-11-06 13:08:04.339282: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-11-06 13:08:04.339511: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-06 13:08:04.339529: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-06 13:08:04.340275: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-06 13:08:04.340647: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
os.makedirs("models", exist_ok=True)
checkpoint = ModelCheckpoint('models/ham10000_model.h5', monitor='val_accuracy', save_best_only=True)



In [25]:
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[checkpoint]
)


Epoch 1/30


2025-11-06 13:08:25.193377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


150/150 [==============================] - ETA: 0s - loss: 2.0551 - accuracy: 0.6444

2025-11-06 13:08:49.672897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [37]:
model.save("ham10000_model_final.h5")

In [38]:
print("✅ Training completed. Model saved as 'ham10000_model.h5'.")


✅ Training completed. Model saved as 'ham10000_model.h5'.


In [39]:
print("Class indices:", train_gen.class_indices)

Class indices: {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
